In [93]:
import importlib
import verse
import corpus
import phonetic
import generate_sample
import numpy as np

In [78]:
importlib.reload(phonetic)

dictionary = phonetic.PhoneticDictionary()

song_files = ["lyrics/kendrick_lamar/GoodKid.txt",
             "lyrics/cardi-b/BodakYellow.txt",
             "lyrics/nicki_minaj/ChunLi.txt",
             "lyrics/drake/NiceForWhat.txt",
             "lyrics/hamilton/processed/AlexanderHamilton.txt"]

unknowns = ["logios_output/kendrick_lamar/GoodKid_logios.txt",
           "logios_output/cardi-b/BodakYellow_logios.txt",
           "logios_output/nicki_minaj/ChunLi_logios.txt",
           "logios_output/drake/NiceForWhat_logios.txt",
           "logios_output/hamilton/AlexanderHamilton_logios.txt"]

kendrick_lamar_verse = "Mass hallucination baby \
Ill education baby \
Want to reconnect with your relations \
This is your station baby \
Look inside these walls and you see them having withdrawals \
Of a prisoner on his way \
Trapped inside your desire \
To fire bullets that stray \
Track a tire just tell you I'm tired and ran away \
I should ask a choir what do you require \
To sing a song that acquire me to have faith \
As the record spin I should pray \
For the record I recognize that I'm easily prey \
I got ate alive yesterday \
I got animosity building \
It's probably big as a building"

cardi_b_verse = "I'm the hottest in the street, know you prolly heard of me \
Got a bag and fixed my teeth, hope you hoes know it ain't cheap \
And I pay my mama bills, I ain't got no time to chill \
Think these hoes be mad at me, their baby father run a bill \
Said, Lil bitch, you can't fuck with me if you wanted to \
These expensive, these is red bottoms, these is bloody shoes \
Hit the store, I can get 'em both, I don't wanna choose \
And I'm quick, cut a nigga off, so don't get comfortable, look"

nicki_minaj_verse = "I went and copped the chopsticks \
Put it in my bun just to pop shit \
I'm always in the top shit \
Box seats, bitch, fuck the gossip \
How many a' them coulda did it with finnesse? \
Now everybody like, She really is the best. \
You play checkers, couldn't beat me playin' chess \
Now I'm about to turn around and beat my chest \
Bitch, it's King Kong, yes, it's King Kong \
Bitch, it's King Kong, this is King Kong \
Chinese ink on, Siamese links on \
Call me 2 Chainz, name go ding dong"

drake_verse = "Everybody get your mothafuckin' roll on \
I know shorty and she doesn't want no slow song \
Had a man last year, life goes on \
Haven't let that thing loose, girl, in so long \
You been inside, know you like to lay low \
I've been peepin' what you bringin' to the table \
Workin' hard, girl, everything paid for \
First, last phone bill, car note, cable \
With your phone out, gotta hit them angles \
With your phone out, snappin' like you Fabo \
And you showin' off, but it's alright \
And you showin' off, but it's alright \
It's a short life, yeah"

hamilton_verse = "How does a bastard, orphan, son of a whore and a \
Scotsman, dropped in the middle of a forgotten \
Spot in the Caribbean by Providence, impoverished, in squalor \
Grow up to be a hero and a scholar? \
The ten-dollar Founding Father without a father \
Got a lot farther by workin’ a lot harder \
By bein’ a lot smarter \
By bein’ a self-starter \
By fourteen, they placed him in charge of a trading charter"

verses = [kendrick_lamar_verse,
         cardi_b_verse,
         nicki_minaj_verse,
         drake_verse,
         hamilton_verse]

In [79]:
def make_template(text):
    template = verse.VerseTemplate(text, dictionary, breakrules="phrase")
    template.get_rhyme()
    return template

In [129]:
def compute_similarity_score(res1, res2):
    
    # iterate by syllable
    # check word at each syllable position and compare between strings
    
    # both templates should have the same number of syllables
    first_template, second_template = res1[1], res2[1]
    num_syls = first_template.num_syllables
    assert num_syls == second_template.num_syllables
    
    total_matches = 0
    for syl_pos in range(num_syls):
        word1 = first_template.wordList[first_template.matrix_indices[syl_pos][0]]
        word2 = second_template.wordList[second_template.matrix_indices[syl_pos][0]]
        # print(word1, word2)
        if word1 == word2:
            total_matches += 1
    # print(total_matches, num_syls)
    return total_matches / num_syls

In [150]:
def intra_similarity(template_text, corpus, unknowns, n_iters):
    
    results = []
    
    for i in range(n_iters):
        template = verse.VerseTemplate(template_text, dictionary,
                              breakrules='phrase')
        template.add_unknowns(unknowns)
        template.get_rhyme()
        result_text = generate_sample.fill_template(corp, template, verbose=True)
        # print("\n", result_text)
        results.append((result_text, template))
        
    score_matrix = np.zeros((len(results), len(results)))
    for i in range(len(results)):
        for j in range(len(results)):
            score_matrix[i,j] = compute_similarity_score(results[i], results[j])
            
    # zero out the diagonal
    np.fill_diagonal(score_matrix, 0)
            
    average_sim_score = np.mean(score_matrix)
    max_sim_score = np.max(score_matrix)
    
    # print("Avg similarity score: ", average_sim_score)
    # print("Max similarity score: ", max_sim_score)
    # print(score_matrix)
    
    return average_sim_score, max_sim_score

In [151]:
importlib.reload(corpus)
importlib.reload(verse)
importlib.reload(generate_sample)

# corp = corpus.WordCorpus(dictionary)

for idx, song in enumerate(song_files):
    
    print("Testing on ", song)
    corp = corpus.WordCorpus(dictionary)
    corp.initialize(song)
    corp.add_unknowns(unknowns[idx])
    corp.initializeMatrix()
    corp.initializeSylDict()
    
    average_sim_scores = []
    max_sim_scores = []

    for i in range(100):
        if i % 20 == 0:
            print("Iteration: ", i + 1)
        avg_sim_score, max_sim_score = intra_similarity(verses[idx], corp, unknowns[idx], 10)

        average_sim_scores.append(avg_sim_score)
        max_sim_scores.append(max_sim_score)
        
        break ## remove
    
    print("Avg similarity score: ", np.sum(average_sim_scores)/len(average_sim_scores))
    print("Max similarity score: ", max(max_sim_scores))


    


Testing on  lyrics/kendrick_lamar/GoodKid.txt
unknowns filename: lyrics/kendrick_lamar/GoodKid_unknowns.txt
Need to add 6 words to template word list using LOGIOS tool
Corpus text: 510 words, 252 unique
Iteration:  1

Fill 19 rhyme pairs:


   Get a match for at syllables 5 and 12
	Both currently empty, pick both words at random
	Get rhymes using the vowel phoneme AH
    => Add ("rhythm" R IH1 DH AH0 M ) and ("bible" B AY1 B AH0 L )

   Get a match for at syllables 5 and 29
	First word hallucination
	Get rhymes using the vowel phoneme AH
    => Add ("victim" V IH1 K T AH0 M )

   Get a match for at syllables 12 and 29
	Occupied: ( / bible  # ) and ( # victim  # )

   Get a match for at syllables 53 and 81
	Both currently empty, pick both words at random
	Get rhymes using the vowel phoneme IH
    => Add ("bill" B IH1 L ) and ("if" IH1 F )

   Get a match for at syllables 60 and 63
	Both currently empty, pick both words at random
	Get rhymes using the vowel phoneme AO
    => Add ("your" 

		Choose between: and
		Filtered scores: [1.]
	 => Add 'and' going forward
	Local state of template: respect the rhythm inspired me in bible study and probably young but that i'm topic is just a victim i quietly hope for trouble and  #  #  #  /  /  # release the matter because the bill 
    to_fill: 6

	Get backwards choices for 'release'
		Choose between: to
		Filtered scores: [1.]
	 => Add 'to' going backward
	Local state of template: respect the rhythm inspired me in bible study and probably young but that i'm topic is just a victim i quietly hope for trouble and  #  #  #  /  / to release the matter because the bill 
    to_fill: 5

	Get forwards choices for 'and'
		Choose between: your, you, blue
		Filtered scores: [0.36 0.37 0.   0.   0.27 0.   0.   0.   0.   0.  ]
	 => Add 'you' going forward
	Local state of template: respect the rhythm inspired me in bible study and probably young but that i'm topic is just a victim i quietly hope for trouble and you  #  #  /  / to release the m

		Choose between: shirt
		Filtered scores: [0. 1.]
	 => Add 'shirt' going backward
	Local state of template: respect the rhythm inspired me in bible study and probably young but that i'm topic is just a victim i quietly hope for trouble and you can we pleasure to release the matter because the bill every time you ever your shirt this and your desire the that i'm easily prey i got time and you wonder if shirt cause now you wonder 
    to_fill: 5

	Get forwards choices for 'time'
		Choose between: you
		Filtered scores: [0. 1.]
	 => Add 'you' going forward
	Local state of template: respect the rhythm inspired me in bible study and probably young but that i'm topic is just a victim i quietly hope for trouble and you can we pleasure to release the matter because the bill every time you ever your shirt this and your desire the that i'm easily prey i got time you you wonder if shirt cause now you wonder 
    to_fill: 4

	Get backwards choices for 'shirt'
		Choose between: your
		Filtered sco

		Choose between: you
		Filtered scores: [0. 1.]
	 => Add 'you' going forward
time you  /  #  / 

	Left, right, to_fill: 150 , 153 , 2
		No plausibles that fit; just add a word

	Get forwards choices for 'you'
		Choose between: respect, wonder, hired, ever
		Filtered scores: [0.   0.   0.   0.   0.47 0.   0.   0.   0.18 0.18 0.18]
	 => Add 'hired' going forward
time you hired  / 

you respect the rhythm inspired me in bible study and probably young but that i'm topic is just a victim i quietly hope for trouble and you can we pleasure to release the matter because the bill every time you ever your shirt this and your desire the that i'm easily prey i got time you ever wonder in this is your bullet you person i got time you wonder if i can we time you ever in all my face from a head is just want to do you ever in all my hindsight that i see in the roof is your station baby look inside your station baby look for the bill every time you hired  / 

 0: you, 1: respect, 3: the, 4: rhythm, 6:

		Choose between: in
		Filtered scores: [0. 0. 1.]
	 => Add 'in' going forward
i see in than 

persevere as a choir what do you i'm by myself promise me standing still i see in than me jumping off of affiliation  #  #  #  /  #  #  #  #  /  /  #  /  #  /  #  /  /  #  # sing  #  /  #  #  / person  #  # acquire  #  #  # acquire  #  #  #  # tired  /  #  / big  #  #  #  / wonder  #  #  #  # fire  #  #  /  #  #  / finding  #  #  #  #  #  /  /  #  #  #  # than  #  /  /  #  #  #  /  #  #  #  #  /  / that  #  #  #  /  #  #  /  #  #  #  #  /  #  /  /  #  /  #  #  /  /  #  #  /  #  / 

FILL BETWEEN 34 AND 52

	Get forwards choices for 'affiliation'
		Choose between: can
		Filtered scores: [1.]
	 => Add 'can' going forward
	Local state of template: as a choir what do you i'm by myself promise me standing still i see in than me jumping off of affiliation can  #  #  /  #  #  #  #  /  /  #  /  #  /  #  /  /  #  # sing 
    to_fill: 18

	Get backwards choices for 'sing'
		Choose between: to
		Filtered

...

Fill (3, 1) syllables

	Left, right, to_fill: 29 , 33 , 3
		No plausibles that fit; just add a word

	Get forwards choices for 'block'
		Choose between: block
		Filtered scores: [1. 0.]
	 => Add 'block' going forward
block block  #  #, oh 

	Left, right, to_fill: 30 , 33 , 2
		No plausibles that fit; just add a word

	Get forwards choices for 'block'
		Choose between: block
		Filtered scores: [1. 0.]
	 => Add 'block' going forward
block block block  #, oh 

	Left, right, to_fill: 31 , 33 , 1
		No plausibles that fit; just add a word

	Get forwards choices for 'block'
		Choose between: block
		Filtered scores: [1. 0.]
	 => Add 'block' going forward
block block block block, oh 

bitches know where i'm the shit, they the hand above my block nope and go to chill, blah yeah your babe my block block block block block block block block block, oh said i'm who ain't got  #  # my  #  #  # club  #,  # what  #  #  /  #  /  #  /, live bags, whip  #  #  #  #  # see  #  #  #  /  #  #  /,  #  /  #  # 

		Choose between: i
		Filtered scores: [1. 0. 0.]
	 => Add 'i' going backward
	Local state of template: know where i'm the shit, they the hand above my block nope and go to chill, blah yeah your babe my block block block block block block block block block, oh said i'm who ain't got no bah my way way way way way, way way way way way way way way way, way way, way way said i'm who they see  #  #  #  /  #  #  /,  #  / i be mad, 
    to_fill: 14

	Get forwards choices for 'way'
		Choose between: way
		Filtered scores: [0. 1.]
	 => Add 'way' going forward
	Local state of template: know where i'm the shit, they the hand above my block nope and go to chill, blah yeah your babe my block block block block block block block block block, oh said i'm who ain't got no bah my way way way way way, way way way way way way way way way, way way, way way way i'm who they see  #  #  #  /  #  #  /,  #  / i be mad, 
    to_fill: 13

	Get backwards choices for 'i'
		Choose between: and, say, ah, yeah
		Filte

		Choose between: horse, opp, rolls, wraith, crib, gate
		Filtered scores: [0.   0.   0.   0.   0.   0.11 0.   0.14 0.11 0.43 0.11 0.11]
	 => Add 'horse' going forward
	Local state of template: know where i'm the shit, they the hand above my block nope and go to chill, blah yeah your babe my block block block block block block block block block, oh said i'm who ain't got no bah my way way way way way, way way way way way way way way way, way way, way way way way way way way way gon bottoms these bitches, know yeah i be mad, at me who they what to who they, tryna be look like, okay who street know you in a horse, yeah only, the street pew  #  #  #  #,  #  /  /  /  # 
    to_fill: 14
		No word exists at fill_index, return current index
	Local state of template: know where i'm the shit, they the hand above my block nope and go to chill, blah yeah your babe my block block block block block block block block block, oh said i'm who ain't got no bah my way way way way way, way way way way way

unknowns filename: lyrics/nicki_minaj/ChunLi_unknowns.txt
Need to add 16 words to template word list using LOGIOS tool
Corpus text: 447 words, 196 unique
Iteration:  1
Need to add 4 words to template word list using LOGIOS tool

Fill 12 rhyme pairs:


   Get a match for at syllables 14 and 21
	Both currently empty, pick both words at random
	Get rhymes using the vowel phoneme EH
    => Add ("drenched" D R EH1 N CH T ) and ("chess" CH EH1 S )

   Get a match for at syllables 36 and 41
	Both currently empty, pick both words at random
	Get rhymes using the vowel phoneme IY
    => Add ("every" EH1 V ER0 IY0 ) and ("she" SH IY1 )

   Get a match for at syllables 65 and 82
	Both currently empty, pick both words at random
	Get rhymes using the vowel phoneme AO
    => Add ("for" F AO1 R ) and ("dong" D AO1 NG )

   Get a match for at syllables 80 and 108
	Both currently empty, pick both words at random
	Get rhymes using the vowel phoneme IH
    => Add ("shit" SH IH1 T ) and ("ding" D IH1 NG )


ValueError: your stub joiner stop condition is fucked up